# Summary Stats
How best to summarize an ice situation in one number?

Brainstorming:
 * Distance to closest ice (perhaps with a minimum threshold so we aren't influenced by camera angle)
 * Distance to farthest ice
 * Percent of frame that is ice, pre-transformation (probably not good because the camera moves all over the place)
 * Percent of ice+water that is ice, pre-transformation (better but still somewhat sensitive to camera movement)
 * Percent of real-space ocean coverage that is ice (same problem plus would be overstating our confidence in ice coverage at the horizon)
 * Percent of log-polar map that is ice (same problem)
 * One of the last three but for a subset of the space that is visible in ~all images (e.g., horizon to 0.25*original height down from the horizon, middle half of the horizontal range)

In the interest of time, let's try out
 1. Distance to closest significant ice
 2. Percent of ice+water within a standard subset that is ice, pre-transformation

We'll use the usual test images:

In [ ]:
from cv_experiments.seg2info import Seg2Info
s2i = Seg2Info()
import cv2

def relevant_details(img, interpolation_method=cv2.INTER_CUBIC):
    img = s2i.one_hot_four(img)
    img = s2i.upscale(img)
    img = s2i.undistort(img)
    horizon = s2i.find_horizon(img)
    img, scale, height = s2i.rotate_image(img, horizon)
    img = s2i.adjust_and_crop(img, horizon, height, interpolation_method=interpolation_method)
    img = s2i.horizon_blur(img)
    lp_img = s2i.camera_to_log_polar(img, scale, height, interpolation_method=interpolation_method)
    lp_img = s2i.four_to_one(lp_img)
    return scale, height, img, lp_img

In [ ]:
images = s2i.load_dirs("../representatives/segmentation/seginput", "../arctic_images_original_2/segmaps")
s2i.apply(relevant_details, images, "segmap", ["scale", "height", "adjusted", "logpolar"])

s2i.plot_all(images, s2i.simple_composite, lambda fig: fig.subplots_adjust(hspace=-0.5))
s2i.plot_key(images, "adjusted")
s2i.plot_all(images, s2i.plot_log_polar, lambda fig: fig.subplots_adjust(hspace=-0.5, wspace=0.4))

## Distance to closest significant ice

In [ ]:
import numpy as np
def closest_ice(logpolar_plot, ice_thresh=0.1, nan_thresh=0.1, close_thresh=None):
    if close_thresh is None: close_thresh = s2i.cam_props["near_distance"]*4
    num = np.nansum(logpolar_plot, axis=1)
    n_nonnan = (~np.isnan(logpolar_plot)).sum(axis=1)
    frac_ice = np.where(n_nonnan/logpolar_plot.shape[1] > nan_thresh, np.divide(num, n_nonnan, where=(n_nonnan != 0)), np.nan)
    is_ice = frac_ice > ice_thresh
    i_last = -np.Inf if np.max(is_ice) == False else logpolar_plot.shape[0]-np.argmax(is_ice[::-1])-1
    dist_last = s2i.y2dist(i_last)
    return np.clip(dist_last, close_thresh, None)

print(s2i.apply(closest_ice, images, "logpolar", "closest_ice"))

In [ ]:
s2i.plot_all(images, lambda ax,image: s2i.plot_log_polar(ax, image, title=f"Closest ice: {image['closest_ice']:.2f} m"), lambda fig: fig.subplots_adjust(hspace=-0.5, wspace=0.4))

Looks good.

## Percent of ice+water within a standard subset that is ice, pre-transformation

In [ ]:
def ice_amount(adjusted, scale, height, window_width_frac=0.5, window_height_frac=0.25):
    window_width_px = window_width_frac*adjusted.shape[1]*scale
    top = s2i.proc_props["sky_buffer"]*s2i.proc_props["upscale_factor"]
    bottom = int(top+window_height_frac*height*scale)
    left = int(adjusted.shape[1]/2-window_width_px/2)
    right = int(adjusted.shape[1]/2+window_width_px/2)
    subset = s2i.four_to_one(adjusted[top:bottom, left:right])

    n_nonnan = (~np.isnan(subset)).sum()  # Exclude invalid data from the denominator
    n_ice = np.nansum(subset)  # Each pixel is iciness from 0 to 1, so we can get total ice in pixels simply by summing
    return n_ice/n_nonnan

print(s2i.apply(ice_amount, images, ["adjusted", "scale", "height"], "ice_amount"))

In [ ]:
s2i.plot_all(images, lambda ax,image: s2i.plot_mask(ax, image, map_key="adjusted", title=f"Ice amount: {image['ice_amount']:.0%}"))

Excellent.
## Meta-summary
The meta-summary with the typical images:

In [ ]:
meta_summary_title = lambda image: f"{image['ice_amount']:.0%} ice, closest at {image['closest_ice']:.2f} m"
s2i.plot_all(images, lambda ax, image: s2i.simple_composite(ax, image, title=meta_summary_title(image)), lambda fig: fig.subplots_adjust(hspace=-0.5))

A scatter plot with all the manually labeled images containing ice:

In [ ]:
import os
icy_set = set(os.path.basename(f) for f in os.listdir("../arctic_images_original_2/ice"))
images2 = s2i.load_dirs("../arctic_images_original_2/seginput", "../arctic_images_original_2/segmaps", filter_fn=lambda f: f in icy_set)
s2i.apply(relevant_details, images2, "segmap", ["scale", "height", "adjusted", "logpolar"]);

In [ ]:
closests = np.array(s2i.apply(closest_ice, images2, "logpolar", "closest_ice"))
amounts = np.array(s2i.apply(ice_amount, images2, ["adjusted", "scale", "height"], "ice_amount"))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
plt.gca().set_xscale("log")
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())
plt.gca().set_ylabel("Ice amount")
plt.gca().set_xlabel("Distance to closest ice")
plt.scatter(closests, amounts);

Filtering out extremes, we get this interesting pattern:

In [ ]:
mask = (closests > s2i.cam_props["near_distance"]*4) & (closests < 1000)
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())
plt.gca().set_ylabel("Ice amount")
plt.gca().set_xlabel("Distance to closest ice")
plt.scatter(closests[mask], amounts[mask]);